In [ ]:
import tensorflow as tf
import keras
import os
import numpy as np
import re
import requests
import tarfile
import shutil
import pandas as pd
import json

from keras.preprocessing.text import Tokenizer, text_to_word_sequence

In [ ]:
# Dataset loading from https://colab.research.google.com/github/CODAIT/project-codenet-notebooks/blob/main/Project_CodeNet_LangClass.ipynb#scrollTo=AdZRI-dTRLhk

file_name = "Project_CodeNet.tar.gz"
data_url = f"https://dax-cdn.cdn.appdomain.cloud/dax-project-codenet/1.0.0/{file_name}"

# Download tar archive to local disk
with open(file_name, "wb") as f:
    f.write(requests.get(data_url).content)
    
# Extract contents of archive to local disk
if os.path.exists("data"):
    shutil.rmtree("data")    
with tarfile.open(file_name) as tf:
    tf.extractall()

KeyboardInterrupt: ignored

In [ ]:
from IPython.core.display import JSON
filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n1234567890'
csv_name = "code_samples.csv"

tokenizer = Tokenizer(
    num_words=10000,
    filters='',
    lower=True, split=' ', char_level=False, oov_token="<UNK>",
    document_count=0
)

if file_name == "Project_CodeNet.tar.gz":
  cd = "Project_CodeNet/data"
  meta_cd = "Project_CodeNet/metadata"
elif file_name == "Mini_Project_CodeNet.tar.gz":
  cd = "data"
  meta_cd = "metadata"

language = "Python"
text = ""
accepted_submissions = []
accepted_samples = pd.DataFrame(columns=["sample","submission_id","status"])
if not os.path.exists(f"drive/MyDrive/ML Pro{csv_name}"):
  accepted_samples.to_csv(f"drive/MyDrive/{csv_name}", index=False)
j = 1

df = pd.read_csv(f"drive/MyDrive/{csv_name}")
num_lines = len(df)
print(num_lines)


for problem in os.listdir(f"{cd}/"):
  #print(j)
  #j+=1
  metadata = pd.read_csv(f"{meta_cd}/{problem}.csv", header=0)
  df = metadata.loc[(metadata["problem_id"] == problem) &  
                 (metadata["language"] == "Python") & 
                 ((metadata["status"] == "Accepted") | 
                  (metadata["status"] == "Rejected"))]
  accepted_submissions += list(df["submission_id"])
  if os.path.exists(f"{cd}/{problem}/{language}"):
    for file_name in sorted(os.listdir(f"{cd}/{problem}/{language}")):
      if j > num_lines:
        with open(f"{cd}/{problem}/{language}/{file_name}", mode='r') as file:
          text = file.read().lower()
          for i in range(len(filters)):
            text = text.replace(filters[i], f" {filters[i]} ")
          texts = text.split(" ")
          texts = [text for text in texts if text != '']
          tokenizer.fit_on_texts(texts)
          if file_name.split(".")[0] in accepted_submissions:
            print("YEET ", j)
            status = df[df["submission_id"] == file_name.split(".")[0]]["status"]
            next_sample = pd.DataFrame(data={"sample":[" ".join(texts)], "submission_id":[file_name.split(".")[0]], "status": [status]})
            try:
              next_sample.to_csv(f"drive/MyDrive/{csv_name}", index=False, header=False, mode="a")
            except:
              print("ERROR OCCURRED WHILE SAVING...")
            j += 1
      else:
        j += 1
        print("YEET_ ", j)


FileNotFoundError: ignored

In [ ]:
print(accepted_samples)
accepted_samples.to_csv("drive/MyDrive/accepted_code_samples.csv", index=False)

In [ ]:
tokenizer_json = tokenizer.to_json()

with open('drive/MyDrive/tokenizer.json', 'w') as f:
    json.dump(tokenizer_json, f)

In [ ]:
tokenizer_json